This notebook measures the intensities and errors of CO 2-1 emission at the locations of all of the SNe in our sample across all of the resolutions in the survey. This is used to prepare to answer the question in section 3.1 Do we see gas at supernova locations? 

We will use this to uncover what resolution will be best for this study.

In [1]:
# IMPORT PACKAGES

#global
import numpy as np
from matplotlib import pyplot as plt
import astropy
import astropy.io.fits as pyfits
from astropy.table import Table, join
from astropy.wcs import WCS
from astropy.io import ascii
from astropy.io import fits
import os
from reproject import reproject_interp

#local
import sys
sys.path.append('/home/mayker.1/Desktop/PythonFunctions')

from findResolution import findRes
from getMapValue    import getValue
from nonZeroError   import findErrVals
from nonZeroError   import nonZeroErrArray
from nonZeroError   import findSignal
from findPercentiles import findStats

In [2]:
# DEFINE FUNCTIONS

def genFileName(galaxy, mapType, res, telOrient):
    
    """
    Generates the filenames for the CO maps.
    
    Parameters
    ----------
    galaxy    : string : name of galaxy
    mapType   : string : "int", "err", or "EW"   
    res       : string : resolution of map ("" (native), "_60pc", "_90pc", "_120pc", "_150pc", etc.)
    telOrient : string : 12m+7m+tp, 7m+tp, etc.
    
    Returns
    -------
    fileName : string : full path to file on Tycho.
        (/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/ngc2997/ngc2997_12m+7m+tp_co21_150pc_broad_mom0.fits)

    """
    
    if mapType == "int":
        mapTypeStr = "_broad_mom0"
    elif mapType == "err":
        mapTypeStr = "_broad_emom0"
    elif mapType  == "EW":
        mapTypeStr = "_strict_ew"
    else:
        print("Wrong Map Type.")
        
    preamble = "/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/" 
    
    fileName = preamble + galaxy + "/" + galaxy + "_" + telOrient + "_co21" + res + mapTypeStr + ".fits"
    return(fileName)


def getAlphaCO(acofileName, intfileName):
    """
    Pulls the metallicity dependent Alpha CO values using Sun+2020 maps
    or assigns a value of alpha CO = 4.35/0.65 if no Sun value is available.
    """

    if(os.path.isfile(acofileName) == True) and (os.path.isfile(intfileName) == True):
        hdu_int = pyfits.open(intfileName)
        hdu_aco = pyfits.open(acofileName)
        acoMap, footprint = reproject_interp(hdu_aco, hdu_int[0].header)

    else:
        #print("NO ACO file: ", acofileName)
        acoMap = 4.35/0.65
    
    return(acoMap)


In [3]:
# import SNeCO data

dataFile = '../Data/3.SNe+GalData.csv'
dataTable = Table.read(dataFile, format='csv') 
#dataTable.colnames

#import SNe data 
SNe_Gal_File = '../Data/3.SNe+GalData.csv'
SNe_Gal_Data = Table.read(SNe_Gal_File, format='csv') 

intenFile = '../Data/3.IntenData.csv'
intenData = Table.read(intenFile, format='csv')

database  = join(SNe_Gal_Data, intenData, keys=('SN_name', 'galaxy'))

In [4]:
# pull galaxy names and telescope orientation to generate the file lists

galaxies  = dataTable['galaxy']
telOrient = dataTable['telOrient']
SNname    = dataTable['SN_name']
SNra      = dataTable['SN_ra']
SNdec     = dataTable['SN_dec']
beamsize  = dataTable['map_beamsize']
telOrient = dataTable['telOrient']
distance  = dataTable['dist']
acoFile   = dataTable['AlphaCOFile']
incl      = dataTable['orient_incl']

In [5]:
# set up resolution array and galaxies without repeats
res = ['60', '90', '120', '150', '500', '750', '1k']
gals = list(dict.fromkeys(galaxies))
acos = list(dict.fromkeys(acoFile))
incls = list(dict.fromkeys(incl))

# find mgsd percentiles of all pixels from galaxy maps
for i in range(len(res)):
    
    resIntPixArr = []

    resStr = "_" + res[i] +  "pc"

    for j in range(len(gals)):
                
        galIntPixArr = []
        
        IntFileName = genFileName(gals[j], "int", resStr, telOrient[j])
        
        if(os.path.isfile(IntFileName) == True):
                        
            hdulist = pyfits.open(IntFileName)
            map = hdulist[0].data
            flatmap=map.flatten()

            #remove nans
            keep  = np.where(np.isfinite(flatmap))
            galIntPixArr = flatmap[keep]
            
            # get alphaco information
            acoMap = getAlphaCO(acos[j], IntFileName)

            if isinstance(acoMap, float) == False:
                acoMap = acoMap.flatten()
                acoMap = acoMap[keep]
            
            galIntPixArr = galIntPixArr * acoMap * np.cos(incls[j]* np.pi/180)

            #add to full list
            resIntPixArr = np.concatenate((galIntPixArr, resIntPixArr), axis=0)
            #print("file for ", gals[j], " at resolution ", res[i])
            
        else:
            pass
            #print("No file for ", galaxies[j], " at resolution ", res[i])
        
    resArr = np.sort(resIntPixArr)
    resCS = np.cumsum(resArr)/np.sum(resArr)
    perArr = np.interp([0.16, 0.5, 0.84],resCS,resArr)
    for k in range(len(perArr)):
        perArr[k] = round(perArr[k],2)
    print("Resolution: ", res[i], " Percentiles: ", perArr)
            


Resolution:  60  Percentiles:  [ 17.09  46.89 201.26]
Resolution:  90  Percentiles:  [ 14.12  42.72 194.38]
Resolution:  120  Percentiles:  [ 13.78  47.9  297.08]
Resolution:  150  Percentiles:  [ 12.97  45.78 274.2 ]
Resolution:  500  Percentiles:  [ 10.01  32.   169.96]
Resolution:  750  Percentiles:  [  9.89  28.96 142.56]
Resolution:  1k  Percentiles:  [  9.93  27.21 120.93]


In [6]:

def percentiles(array):

    vals = np.asarray(array)
    
    for i in range(len(vals)):
        if vals[i] < 0.0:
            vals[i] = 0.0
    
    vals = vals[np.isfinite(vals)]
    
    percentile = np.linspace(0,1,len(vals))
    stats = findStats(percentile, vals)

    return(stats, len(vals))

def mgsdPercentiles(res):

    msgdStr = "pc" + str(res) + "mgsd"
    errStr = "pc" + str(res) + "mgsdErr"

    mgsdgroup = database.group_by([msgdStr, errStr])

    mgsdSort, errSort = [],[]
    
    # remove nans and set negative values to zero
    for i in range(len(mgsdgroup)):
        if np.isfinite(mgsdgroup[msgdStr][i]):
#             if mgsdgroup[msgdStr][i] < 0:
#                 mgsdgroup[msgdStr][i] = 0.0
#             if mgsdgroup[errStr][i] < 0:
#                 mgsdgroup[errStr][i] = 0.0
            mgsdSort.append(mgsdgroup[msgdStr][i])
            errSort.append(mgsdgroup[errStr][i])
        else:
            pass

    # find normalized cumulative dictribution    
    mgsdSort = np.sort(mgsdSort)
    errSort = np.sort(errSort)
    mgsdCS = np.cumsum(mgsdSort)/np.sum(mgsdSort)
    errCS = np.cumsum(errSort)/np.sum(errSort)

    # find three sigmas
    percentiles = np.interp([0.16,0.5,0.84],mgsdCS,mgsdSort)
    for i in range(len(percentiles)):
        percentiles[i] = round(percentiles[i],2)
    print(res, len(mgsdSort), percentiles)


In [7]:
# Find entries for Latex Table : MGSD Percentiles at all resolutions at locations of SNe

pc60mgsdTiles = mgsdPercentiles(60)
pc90mgsdTiles = mgsdPercentiles(90)
pc120mgsdTiles = mgsdPercentiles(120)
pc150mgsdTiles = mgsdPercentiles(150)
pc500mgsdTiles = mgsdPercentiles(500)
pc750mgsdTiles = mgsdPercentiles(750)
pc1000mgsdTiles = mgsdPercentiles(1000)


60 7 [128.56 324.77 548.54]
90 19 [ 87.94 293.84 583.09]
120 47 [ 70.3  325.93 789.23]
150 59 [ 76.21 317.94 740.26]
500 62 [ 60.96 216.03 645.07]
750 63 [ 57.95 209.31 545.31]
1000 63 [ 46.17 164.36 474.16]
